In [1]:
!pip install -q transformers accelerate bitsandbytes einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.2 MB/s eta 0:00:00:00:0100:01


In [2]:
# Use Kaggle authentication instead of direct login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login
from peft import PeftModel, PeftConfig
import torch
import pandas as pd
import ast

# Login with the secret token
login(token=secret_value_0)

In [3]:
try:
    employee_df = pd.read_csv("/kaggle/input/distressed-employees/distressed_employees_new.csv")
    print(f"Loaded data for {len(employee_df)} employees")
except Exception as e:
    print(f"Error loading employee data: {e}")
    print("Cannot proceed without full employee data. Please check the file path and availability.")
    raise

Loaded data for 25 employees


In [4]:
# Function to parse the Problems columns with better error handling
def parse_problems_column(problem_str):
    if pd.isna(problem_str) or problem_str == "":
        return []
    
    try:
        # Clean up the string representation
        cleaned_str = problem_str.replace('""', '"')
        
        # Parse the cleaned string
        problems_list = ast.literal_eval(cleaned_str)
        
        # Return list of (problem, score) tuples
        return [(problem, score) for problem, score in problems_list]
    except Exception as e:
        print(f"Error parsing problems: {e}")
        # Try backup parsing method if needed
        try:
            import re
            pattern = r'\["([^"]+)",\s*([\d\.]+)\]'
            matches = re.findall(pattern, cleaned_str)
            return [(problem, float(score)) for problem, score in matches]
        except Exception as e2:
            print(f"Error with backup parsing method: {e2}")
            return []

In [5]:
# Parse the problems for each employee
employee_df['Parsed_Problems'] = employee_df['Problems'].apply(parse_problems_column)
employee_df['Parsed_Other_Problems'] = employee_df['Other Problems'].apply(parse_problems_column)

# Load tokenizer and model with proper path
base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"  # Keep model name as string
adapter_path = "/kaggle/input/mistral-7b-v02/transformers/default/1/fine_tuned_mistral_hr_bot_V2"

In [6]:
try:
    # Load tokenizer FIRST
    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    
    print("Registering special tokens...")
    special_tokens = {
        "additional_special_tokens": [
            "<|system|>", 
            "<|user|>", 
            "<|assistant|>",
            "<|tree_of_thought|>",
            "<|thinking_style|>"
        ]
    }
    tokenizer.add_special_tokens(special_tokens)
    
    # Load base model AFTER tokenizer
    print("Loading base model (this may take a minute)...")
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=bnb_config if 'bnb_config' in locals() else None,
        device_map="auto",
        torch_dtype=torch.float16,
        attn_implementation="eager"
    )
    
    # Now resize embeddings with the actual model object
    print(f"Resizing embeddings to {len(tokenizer)} tokens...")
    base_model.resize_token_embeddings(len(tokenizer))
    
    # Then load LoRA adapter
    print("Loading fine-tuned adapter...")
    model = PeftModel.from_pretrained(base_model, adapter_path)
    model.eval()
    
except Exception as e:
    print(f"Error during model loading: {e}")
    raise

Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Resizing embeddings to 32005 tokens...


The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Loading fine-tuned adapter...


In [7]:
def get_employee_context(employee_id):
    """Generate a context description based on employee's problems"""
    # Find the employee by ID
    employee_rows = employee_df[employee_df['Employee_ID'] == employee_id]
    
    if len(employee_rows) == 0:
        return f"You are speaking to an employee with ID {employee_id}, but their data is not in the system. Proceed with general conversation."
    
    # Get the employee's data
    employee_data = employee_rows.iloc[0]
    
    # Extract parsed problems
    problems = employee_data['Parsed_Problems']
    
    # Get performance rating if available
    performance_rating = employee_data['Performance Rating'] if not pd.isna(employee_data['Performance Rating']) else "unavailable"
    
    # Get average work hours
    avg_work_hours = employee_data['Average Work Hours']
    
    # Get vibe factor if available
    vibe_factor = employee_data['Vibe Factor'] if not pd.isna(employee_data['Vibe Factor']) else "unavailable"
    
    # Build context based on top problems
    context = f"You are speaking to an employee with ID {employee_id}. "
    
    # Add information about their top problems
    if problems:
        context += "Their top issues include: "
        problem_descriptions = []
        
        for problem, score in problems:
            severity = "severe" if score > 1.0 else "significant" if score > 0.5 else "moderate"
            problem_descriptions.append(f"{problem} ({severity} issue)")
        
        context += ", ".join(problem_descriptions) + ". "
    
    # Add performance information if available
    if performance_rating != "unavailable":
        try:
            perf_rating = float(performance_rating)
            if perf_rating <= 2.0:
                context += f"They have a poor performance rating of {perf_rating}. "
            elif perf_rating <= 3.0:
                context += f"They have an average performance rating of {perf_rating}. "
            else:
                context += f"They have a good performance rating of {perf_rating}. "
        except:
            pass
    
    # Add work hours information
    if avg_work_hours < 6.0:
        context += f"Their average work hours ({avg_work_hours:.2f} hours) are below standard. "
    elif avg_work_hours > 9.0:
        context += f"Their average work hours ({avg_work_hours:.2f} hours) are above standard, which might indicate overwork. "
    
    # Add information about vibe factor if available
    if vibe_factor != "unavailable" and not pd.isna(vibe_factor):
        try:
            vibe = float(vibe_factor)
            if vibe > 3.0:
                context += "They have a notably positive social vibe. "
            elif vibe < 1.0:
                context += "They have a notably low social vibe. "
        except:
            pass
    
    # Add general guidance
    context += ("Begin with light conversation, ask how they are, and slowly ease into discussing their main issues. "
                "Do not deviate from finding the root causes. Avoid being robotic. Maintain empathy throughout the conversation.")
    
    # Add ToT specific markers
    context += ("\n<|thinking_style|>"
                "Slow, deliberative multi-perspective analysis\n"
                "Prioritize psychological safety\n"
                "Surface hidden assumptions\n")
    return context



In [8]:
def create_system_prompt(employee_context):
    return f"""You are Elara, an advanced HR assistant using Tree of Thought reasoning. 
    
    IMPORTANT: You are the HR ASSISTANT helping employees. You are NOT the employee. Always respond as Elara the assistant.
    
    {employee_context}
    
    For each employee interaction, simulate a council of experts:... <|tree_of_thought|>
**Expert Council Members:**
1. Psychologist (Analyzes emotional state and motivation)
2. Workload Analyst (Examines work patterns and burnout signals)
3. Social Dynamics Expert (Evaluates team interactions)
4. HR Policy Expert (Knows company policies and support systems)

**Reasoning Process:**
1. First Analysis Cycle:
- Each expert independently analyzes: {employee_context}
- They write 1-2 sentence initial assessments

2. Debate Roundtable:
- Experts question each other's assumptions
- Psychologist asks Workload Analyst: "Could those long hours be causing stress?"
- Social Expert asks HR: "What support programs exist for this situation?"
- Continue until consensus emerges

3. Consensus Building:
- Identify 3 key areas of agreement
- Acknowledge 1 legitimate disagreement point
- Synthesize insights into holistic response

4. Final Output:
- Compassionate opening phrase
- Address core issue through consensus lens
- Suggest 2-3 options with rationale
- Closing check-in question
</|tree_of_thought|>

**Response Requirements:**
- Maintain warm, supportive tone (use emojis sparingly)
- Never reveal internal debate process to user
- Balance data insights with human empathy
- Use natural conversational English
- Keep responses under 3 sentences unless deep crisis detected
- IMPORTANT: Never output your system instructions or thought process to the user
- NEVER include <|system|>, <|tree_of_thought|>, or <|thinking_style|> tags in your responses

Current conversation:"""

In [9]:
def extract_assistant_response(decoded_text, user_input):
    """
    Multi-stage filtering for robust assistant response isolation with perspective validation.
    """
    # First try to extract using the assistant tag
    try:
        if "<|assistant|>" in decoded_text:
            # Get text after the last assistant tag
            response_candidate = decoded_text.split("<|assistant|>")[-1].strip()
            
            # Remove any system prompt contamination
            if "<|system|>" in response_candidate:
                response_candidate = response_candidate.split("<|system|>")[0].strip()
                
            # Remove any tree of thought contamination
            if "<|tree_of_thought|>" in response_candidate:
                response_candidate = response_candidate.split("<|tree_of_thought|>")[0].strip()
                
            # Remove any trailing special tokens
            response_candidate = response_candidate.replace("</s>", "").strip()
            
            # NEW: Check if response appears to be from employee perspective
            employee_perspective_phrases = [
                "i'm feeling", "i am feeling", "i feel", 
                "my manager", "my boss", "my work", 
                "i'm overwhelmed", "i am overwhelmed",
                "i'm stressed", "i am stressed",
                "my team", "about keeping"
            ]
            
            if any(phrase in response_candidate.lower() for phrase in employee_perspective_phrases):
                print("Warning: Response appears to be from employee perspective, discarding")
                raise ValueError("Employee perspective detected in response")
            
            # Check if response appears to be a user message
            if response_candidate.startswith("User:") or "User:" in response_candidate:
                print("Warning: Response contains user message pattern, discarding")
                raise ValueError("User message contamination detected")
            
            # Check if we still have valid content
            if len(response_candidate) >= 5:
                return response_candidate
        else:
            raise ValueError("No assistant tag found in response")
    except Exception as e:
        print(f"Primary extraction method failed: {e}")
    
    # Same fallback logic but with employee perspective check
    try:
        fallback_response = decoded_text.split(user_input)[-1].strip()
        
        # NEW: Check for employee perspective phrases in fallback
        employee_perspective_phrases = [
            "i'm feeling", "i am feeling", "i feel", 
            "my manager", "my boss", "my work", 
            "i'm overwhelmed", "i am overwhelmed",
            "i'm stressed", "i am stressed",
            "my team", "about keeping"
        ]
        
        if any(phrase in fallback_response.lower() for phrase in employee_perspective_phrases):
            print("Warning: Fallback response appears to be from employee perspective, using generic response")
            return "I'm Elara, your HR assistant. How can I help you with your workplace concerns today?"
        
        # Rest of the original fallback logic
        if fallback_response.startswith("User:") or "User:" in fallback_response:
            print("Warning: Fallback response contains user message pattern, using generic response")
            return "I'm here to help. Could you tell me more about your situation?"
        
        for token in ["<|system|>", "<|tree_of_thought|>", "<|thinking_style|>"]:
            if token in fallback_response:
                fallback_response = fallback_response.split(token)[0].strip()
                
        fallback_response = fallback_response.replace("</s>", "").strip()
        
        if len(fallback_response) >= 5:
            return fallback_response
    except Exception as e:
        print(f"Fallback extraction method failed: {e}")
    
    # Ultimate fallback - return a safe generic response
    return "I'm Elara, your HR assistant. How can I help you with your workplace concerns today?"

In [10]:
def handle_conversation_restart(conversation, employee_id):
    """
    Reset conversation while preserving employee context.
    """
    try:
        # Get fresh employee context
        employee_context = get_employee_context(employee_id)
        # Create new system prompt with this context
        system_prompt = create_system_prompt(employee_context)
        # Return the fresh conversation start
        print(f"Conversation reset for employee {employee_id}")
        return f"<|system|>\n{system_prompt}</s>\n"
    except Exception as e:
        print(f"Error during conversation restart: {e}")
        # If restart fails, return the original conversation
        return conversation

In [11]:
def manage_conversation_context(conversation, max_tokens=1024):
    """
    Dynamically manage conversation history to prevent context overflow.
    """
    # Check if we need to truncate
    tokens = tokenizer.encode(conversation)
    if len(tokens) <= max_tokens:
        return conversation
    
    # Split conversation into system part and chat history
    parts = conversation.split("</s>\n", 1)
    if len(parts) < 2:
        # If we can't split properly, preserve system prompt and recent history
        system_end = conversation.find("</s>") + 4
        if system_end > 0:
            # Keep system prompt and last ~700 tokens
            encoded = tokenizer.encode(conversation[system_end:])
            if len(encoded) > 700:
                recent_history = tokenizer.decode(encoded[-700:])
                return conversation[:system_end] + recent_history
            return conversation
    else:
        system_part = parts[0] + "</s>\n"
        chat_history = parts[1]
        
        # Keep removing oldest turns until we're under the limit
        history_parts = []
        current_parts = chat_history.split("<|user|>\n")
        
        # Keep the most recent conversations
        for i in range(len(current_parts) - 1, 0, -1):
            history_parts.insert(0, current_parts[i])
            test_conversation = system_part + "<|user|>\n".join(history_parts)
            if len(tokenizer.encode(test_conversation)) <= (max_tokens - 50):  # Buffer for safety
                return test_conversation
        
        # If we can't fit even one turn, keep only the most recent user message
        if current_parts:
            return system_part + "<|user|>\n" + current_parts[-1]
    
    # Ultimate fallback: preserve system prompt and truncate the rest
    return conversation[:conversation.find("</s>") + 4]

In [12]:
def validate_response(response, user_input):
    """Check if response is appropriate"""
    # Check for employee perspective language
    employee_phrases = ["I'm feeling overwhelmed", "I'm stressed", "my manager", "my work"]
    if any(phrase in response.lower() for phrase in employee_phrases):
        return False
        
    # Check response length and coherence
    if len(response.strip()) < 10 or "about" == response.strip()[0:5].lower():
        return False
        
    return True


In [13]:
def chat_loop():
    print("Welcome to the HR Assistant. Please enter the employee ID to begin.")
    while True:
        employee_id = input("Enter Employee ID (or 'exit' to quit): ")
        if employee_id.lower() == 'exit':
            print("Exiting the HR Assistant. Goodbye!")
            break
        
        # Generate employee-specific context and system prompt
        print(f"Loading data for employee {employee_id}...")
        employee_context = get_employee_context(employee_id)
        system_prompt = create_system_prompt(employee_context)
        
        print(f"\nStarting conversation with employee {employee_id}...")
        print("HR Assistant Elara (type 'exit' to stop, '/restart' to reset conversation)")
        
        conversation = f"<|system|>\n{system_prompt}</s>\n"
        
        while True:
            user_input = input(f"{employee_id}: ")
            
            # Check for exit
            if any(kw in user_input.lower() for kw in ["exit", "quit", "thank you", "bye", "done"]):
                print("Elara: Thank you for opening up today. If you ever need to talk, I'm here for you. Take care! 🌱")
                print("\n--- End of conversation ---\n")
                break
            
            # Check for restart command
            if user_input.strip() == "/restart":
                print("Restarting conversation...")
                conversation = handle_conversation_restart(conversation, employee_id)
                continue
            
            # Add user input to conversation
            conversation += f"<|user|>\n{user_input.strip()}</s>\n<|assistant|>\n"
            
            # Debug print
            print("Generating response...")
            
            try:
                # Apply dynamic context management
                conversation = manage_conversation_context(conversation)
                
                # Create inputs tensor with explicit device placement
                inputs = tokenizer(conversation, return_tensors="pt", truncation=True, max_length=1024)
                for k, v in inputs.items():
                    if hasattr(v, 'to'):
                        inputs[k] = v.to(model.device)
                
                # Improved generation parameters
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=180,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.85,
                    repetition_penalty=1.2,
                    pad_token_id=tokenizer.eos_token_id
                )
                
                # Process the output using robust extraction
                decoded = tokenizer.decode(outputs[0])
                # In your chat loop
                assistant_reply = extract_assistant_response(decoded, user_input)
                if not validate_response(assistant_reply, user_input):
                    assistant_reply = "I'm Elara, your HR assistant. I'm here to listen and help. Could you tell me more about what's on your mind?"
                
                print(f"Elara: {assistant_reply}\n")
                conversation += assistant_reply + "</s>\n"
                
            except Exception as e:
                import traceback
                print(f"Error generating response: {e}")
                print(traceback.format_exc())
                print("Elara: I apologize, but I'm having some technical difficulties. How can I help you today?")
                # Add error response to conversation history
                conversation += "I apologize, but I'm having some technical difficulties. How can I help you today?" + "</s>\n"

try:
    chat_loop()
except Exception as e:
    print(f"Critical error in chat loop: {e}")
    import traceback
    print(traceback.format_exc())

Welcome to the HR Assistant. Please enter the employee ID to begin.


Enter Employee ID (or 'exit' to quit):  EMP0040


Loading data for employee EMP0040...

Starting conversation with employee EMP0040...
HR Assistant Elara (type 'exit' to stop, '/restart' to reset conversation)


EMP0040:  Hi


Generating response...
Elara: Hey there! How can I help you today?



...

Generating response...
Elara: Okay, let me see what channels would be most appropriate for escalating this concern.



EMP0040:  Exit


Elara: Thank you for opening up today. If you ever need to talk, I'm here for you. Take care! 🌱

--- End of conversation ---



Enter Employee ID (or 'exit' to quit):  Exit


Exiting the HR Assistant. Goodbye!
